In [1]:
import pandas as pd
import sqlalchemy as sa  # Corrigido
from sqlalchemy import create_engine  # Importação correta para criar o engine
import os
import sys
sys.path.append(r'c:\Projetos\Python\JornadaDeDados')

In [2]:
df = pd.read_csv("microdados_enem_2023/DADOS/MICRODADOS_ENEM_2023.csv", delimiter=";", encoding="latin-1")

In [3]:
df.head()

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210059085136,2023,14,M,2,1,1,1,17,1,...,C,C,B,B,A,B,B,A,A,B
1,210059527735,2023,12,M,2,1,0,1,16,1,...,B,A,B,B,A,A,C,A,D,B
2,210061103945,2023,6,F,1,1,1,1,0,1,...,B,A,A,B,A,A,A,A,A,B
3,210060214087,2023,2,F,1,3,1,2,0,2,...,A,A,A,B,A,A,D,A,A,B
4,210059980948,2023,3,F,1,3,1,2,0,2,...,A,A,A,B,A,A,B,A,A,A


In [4]:
df.columns.values

array(['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE',
       'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO',
       'IN_TREINEIRO', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC',
       'CO_UF_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC',
       'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'CO_MUNICIPIO_PROVA',
       'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 'SG_UF_PROVA',
       'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'TP_PRESENCA_MT', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
       'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
       'NU_NOTA_MT', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH',
       'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TP_LINGUA',
       'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC',
       'TX_GABARITO_MT', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1',
       'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5',
       'NU_NOTA_REDACAO', 'Q001', 'Q002', '

In [5]:
dadosLocalAplicacaoProva = [
    'CO_MUNICIPIO_PROVA',
    'NO_MUNICIPIO_PROVA', 
    'CO_UF_PROVA', 
    'SG_UF_PROVA']

In [6]:
dadosLocalAplicacaoProva = df.filter(items=dadosLocalAplicacaoProva)

In [7]:
dadosLocalAplicacaoProva.head()

,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA
0,5300108,Brasília,53,DF
1,5300108,Brasília,53,DF
2,4305108,Caxias do Sul,43,RS
3,2304400,Fortaleza,23,CE
4,2311306,Quixadá,23,CE


In [8]:
dadosAplicacaoProva = dadosLocalAplicacaoProva.drop_duplicates()

In [9]:
# Removendo registros onde toda a linha está como NaN
dadosLimpos = dadosAplicacaoProva.dropna(how='all')

dadosLimpos.head()

,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA
0,5300108,Brasília,53,DF
2,4305108,Caxias do Sul,43,RS
3,2304400,Fortaleza,23,CE
4,2311306,Quixadá,23,CE
5,2913606,Ilhéus,29,BA


In [18]:
dadosLimpos.dtypes

CO_MUNICIPIO_PROVA             Int64
NO_MUNICIPIO_PROVA    string[python]
CO_UF_PROVA                    Int64
SG_UF_PROVA           string[python]
dtype: object

In [27]:
dadosLimpos = dadosLimpos.convert_dtypes()
for col in dadosLimpos.select_dtypes(include=['object']).columns:
    dadosLimpos[col] = dadosLimpos[col].astype(str)

# Verificar os tipos antes da inserção
print(dadosLimpos.dtypes)

CO_MUNICIPIO_PROVA             Int64
NO_MUNICIPIO_PROVA    string[python]
CO_UF_PROVA                    Int64
SG_UF_PROVA           string[python]
dtype: object


In [21]:
#Descobrindo o tamanho maximo do campo para criar a tabela de forma correta.
dadosLimpos['SG_UF_PROVA'].fillna('').str.len().max()

np.int64(2)

In [41]:
# Remover duplicatas mantendo apenas a primeira ocorrência
dadosLimpos = dadosLimpos.drop_duplicates()

# Exibir os dados após a remoção de duplicatas
print("📌 Após remoção de duplicatas:")
print(dadosLimpos.dtypes)
print(dadosLimpos.head())  # Exibe as primeiras linhas



📌 Após remoção de duplicatas:
CO_MUNICIPIO_PROVA             Int64
NO_MUNICIPIO_PROVA    string[python]
CO_UF_PROVA                    Int64
SG_UF_PROVA           string[python]
dtype: object
   CO_MUNICIPIO_PROVA NO_MUNICIPIO_PROVA  CO_UF_PROVA SG_UF_PROVA
0             5300108           Brasília           53          DF
2             4305108      Caxias do Sul           43          RS
3             2304400          Fortaleza           23          CE
4             2311306            Quixadá           23          CE
5             2913606             Ilhéus           29          BA


In [45]:
import pyodbc
import pandas as pd
from datetime import datetime
from app.db_config import database_config

# Criar a string de conexão diretamente para `pyodbc`
conn_str = (
    f"DRIVER=ODBC Driver 17 for SQL Server;"
    f"SERVER={database_config['host']},{database_config['port']};"
    f"DATABASE={database_config['database']};"
    f"UID={database_config['username']};"
    f"PWD={database_config['password']};"
    "TrustServerCertificate=yes"
)

# Criar a conexão diretamente com `pyodbc`
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Ler os dados existentes na tabela
query = "SELECT CO_MUNICIPIO_PROVA, NO_MUNICIPIO_PROVA, CO_UF_PROVA, SG_UF_PROVA FROM DadosLocalAplProva WHERE is_active = 1"
df_existente = pd.read_sql(query, conn)

# Remover duplicatas no novo dataset
dadosLimpos = dadosLimpos.drop_duplicates()

# Identificar os registros que precisam ser atualizados (já existem e mudaram)
df_merge = df_existente.merge(dadosLimpos, on=['CO_MUNICIPIO_PROVA', 'CO_UF_PROVA'], suffixes=('_old', '_new'), how='inner')

df_alterados = df_merge[df_merge['NO_MUNICIPIO_PROVA_old'] != df_merge['NO_MUNICIPIO_PROVA_new']]

# Fechar os registros antigos (set `end_date`)
for _, row in df_alterados.iterrows():
    cursor.execute(
        "UPDATE DadosLocalAplProva SET DATA_FIM = ?, is_active = 0 WHERE CO_MUNICIPIO_PROVA = ? AND CO_UF_PROVA = ? AND is_active = 1",
        (datetime.now(), row['CO_MUNICIPIO_PROVA'], row['CO_UF_PROVA'])
    )

# Criar query de inserção dos novos registros com `start_date`
insert_query = """
    INSERT INTO DadosLocalAplProva (CO_MUNICIPIO_PROVA, NO_MUNICIPIO_PROVA, CO_UF_PROVA, SG_UF_PROVA, DATA_INICIO, is_active)
    VALUES (?, ?, ?, ?, ?, ?)
"""

# Converter DataFrame para lista de tuplas
records = [tuple(x) + (datetime.now(), 1) for x in dadosLimpos.to_numpy()]

# Inserir os novos registros
cursor.executemany(insert_query, records)

# Confirmar a transação
conn.commit()

print("✅ SCD Tipo 2 aplicado com sucesso!")

# Fechar a conexão
cursor.close()
conn.close()


✅ SCD Tipo 2 aplicado com sucesso!


C:\Users\Thaan\AppData\Local\Temp\ipykernel_11628\2346739767.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_existente = pd.read_sql(query, conn)
